# Example: QAOA as QiskitPattern

This tutorial will be demonstation of creating QAOA as QiskitPattern as well as migration guide on how you can replicate IBM Quantum QAOA custom runtime program.

Let's first get information on what is QAOA runtime program and what inputs and outputs it has. We will not be implementing full set of input/outputs, but we will cover most important ones. Later on we can recover full functionality if needed.

**Description**: Qiskit Runtime QAOA program.


**Inputs**:

| name | type | description |
| ---- | ---- | ----------- |
|initial_point|[array,string]|Initial parameters of the ansatz. Can be an array or the string ``'random'`` to choose random initial parameters. The type must be numpy.ndarray or str.|
|operator|object|The cost Hamiltonian, consisting of Pauli I and Z operators, whose smallest eigenvalue we're trying to find. The type must be a PauliSumOp.|
|method|str|The classical optimizer used to update the parameters in each iteration. Per default, COBYLA|
|ansatz|QuantumCircuit|Ansatz for optimization|

**Return values**

| name | type | description |
| ---- | ---- | ----------- |
|optimal_point|array|The optimal parameter values found during the optimization.|
|optimal_value|number|The smallest value found during the optimization. Equal to the ``eigenvalue`` attribute.|

We will also add optional `QiskitRuntimeService` as an argument to use that to access real devices.


With that information we can start drafting our pattern implementation in `qaoa.py` file.

What our pattern should do:

1. parse input arguments
2. create run_qaoa function that accepts estimator instance, creates VQE and runs calculation
3. decide which sampler to use and run vqe
    - if runtime service was passed then create a session and run `run_qaoa` function
    - if runtime service was not passed then use stantard qiskit sampler
4. save results from qaoa

Roughly our QAOA pattern will look like this. Full code can be found in [qaoa.py](./source_files/qaoa/qaoa.py) file.

```python
# qaoa.py

import ...

def run_qaoa(
    ansatz: QuantumCircuit,
    estimator: BaseEstimator,
    operator: PauliSumOp,
    initial_point: np.array,
    method: str
):
    return minimize(cost_func, initial_point, args=(ansatz, operator, estimator), method=method)


arguments = get_arguments()
service = arguments.get("service")
operator = arguments.get("operator")
initial_point = arguments.get("initial_point")
ansatz = arguments.get("ansatz", 1)
...
if service is not None:
    # if we have service we need to open a session and create sampler
    service = arguments.get("service")        
    backend = arguments.get("backend", "ibmq_qasm_simulator")
    with Session(service=service, backend=backend) as session:
        estimator = Estimator(session=session, options=options)
else:
    # if we do not have a service let's use standart local sampler
    estimator = QiskitEstimator()

result = run_qaoa(ansatz, estimator, operator, initial_point, "COBYLA")
    
save_result({
    "optimal_point": result.x.tolist(),
    "optimal_value": result.fun
})
```

At this point we have our pattern implemented. Now we need to actually run it. But before let's prepare input arguments from our QAOA pattern.

In [ ]:
from qiskit.circuit.library import qaoa_ansatz
from qiskit.quantum_info import SparsePauliOp

from qiskit_ibm_runtime import QiskitRuntimeService


operator = SparsePauliOp.from_list(
    [("IIIZZ", 1), ("IIZIZ", 1), ("IZIIZ", 1), ("ZIIIZ", 1)]
)
ansatz = qaoa_ansatz(operator, reps=2)
ansatz.draw(fold=-1)

┌───┐                                                                            ┌──────────────┐                                                                            ┌──────────────┐
q_0: ┤ H ├─■──────────────■──────────────────────────────■──────────────■─────────────┤ Rx(2.0*β[0]) ├─■──────────────■──────────────────────────────■──────────────■─────────────┤ Rx(2.0*β[1]) ├
     ├───┤ │ZZ(2.0*γ[0])  │             ┌──────────────┐ │              │             └──────────────┘ │ZZ(2.0*γ[1])  │             ┌──────────────┐ │              │             └──────────────┘
q_1: ┤ H ├─■──────────────┼─────────────┤ Rx(2.0*β[0]) ├─┼──────────────┼──────────────────────────────■──────────────┼─────────────┤ Rx(2.0*β[1]) ├─┼──────────────┼─────────────────────────────
     ├───┤                │ZZ(2.0*γ[0]) ├──────────────┤ │              │                                             │ZZ(2.0*γ[1]) ├──────────────┤ │              │                             
q_2: ┤ H ├────────────────■─────────────┤ Rx(2.0*β[0]) ├─┼──────────────┼─────────────────────────────────────────────■─────────────┤ Rx(2.0*β[1]) ├─┼──────────────┼─────────────────────────────
     ├───┤                              └──────────────┘ │ZZ(2.0*γ[0])  │             ┌──────────────┐                              └──────────────┘ │ZZ(2.0*γ[1])  │             ┌──────────────┐
q_3: ┤ H ├───────────────────────────────────────────────■──────────────┼─────────────┤ Rx(2.0*β[0]) ├───────────────────────────────────────────────■──────────────┼─────────────┤ Rx(2.0*β[1]) ├
     ├───┤                                                              │ZZ(2.0*γ[0]) ├──────────────┤                                                              │ZZ(2.0*γ[1]) ├──────────────┤
q_4: ┤ H ├──────────────────────────────────────────────────────────────■─────────────┤ Rx(2.0*β[0]) ├──────────────────────────────────────────────────────────────■─────────────┤ Rx(2.0*β[1]) ├
     └───┘                                                                            └──────────────┘                                                                            └──────────────┘

In [2]:
USE_RUNTIME_SERVICE = False

service = None
if USE_RUNTIME_SERVICE:
    service = QiskitRuntimeService(
        channel="ibm_quantum", instance="ibm-q/open/main", token="<insert your token>"
    )
backend = None

input_arguments = {
    "initial_point": None,
    "ansatz": ansatz,
    "operator": operator,
    "service": service,
    "backend": backend,
}
input_arguments

{'initial_point': None,
 'ansatz': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7ff15980d490>,
 'operator': SparsePauliOp(['IIIZZ', 'IIZIZ', 'IZIIZ', 'ZIIIZ'],
               coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j]),
 'service': None,
 'backend': None}

In [3]:
from qiskit_serverless import ServerlessClient
import os

In [4]:
serverless = ServerlessClient(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)
serverless

<gateway-client>

In [30]:
from qiskit_serverless import QiskitFunction

function = QiskitFunction(
    title="qaoa",
    entrypoint="qaoa.py",
    working_dir="./source_files/qaoa/",
    dependencies=["qiskit_aer"],
)

serverless.upload(function)

QiskitFunction(qaoa)

In [31]:
job = serverless.run("qaoa", arguments=input_arguments)
job

<Job | f8e54d5e-1ccc-44a0-96c2-fba17b7bc9b0>

In [40]:
job.status()

'DONE'

In [42]:
job.result()

{'optimal_point': [6.797265869454594,
  5.057364852138085,
  5.365018757316608,
  3.5613069863037126],
 'optimal_value': -3.1924,
 'probabilitie': {'00001': 4139,
  '11110': 4145,
  '11010': 56,
  '01001': 52,
  '10101': 18,
  '11100': 58,
  '10000': 122,
  '01111': 107,
  '00100': 115,
  '01000': 109,
  '11111': 79,
  '11101': 117,
  '10011': 13,
  '00000': 94,
  '10110': 64,
  '10111': 109,
  '11011': 112,
  '00011': 52,
  '01010': 18,
  '00010': 122,
  '10001': 54,
  '00101': 53,
  '00111': 17,
  '01110': 47,
  '11000': 14,
  '10010': 18,
  '00110': 16,
  '01011': 19,
  '11001': 14,
  '10100': 11,
  '01100': 18,
  '01101': 18}}